In [10]:
import numpy as np
import pandas as pd
import requests

In [11]:
# Imports the csv file with all indicators
indicator_list = pd.read_csv('indicators.csv')
indicator_list = indicator_list.drop(indicator_list.index[-1])
indicator_list.columns = ['Indicator', 'Description']

In [19]:
requests.get(f'{url}CompactData')

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [16]:
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'CompactData/FM/A.GB.'


data = (requests.get(f'{url}{key}{indicator_list.iloc[0, 0]}').json()
        ['CompactData']['DataSet']['Series'])

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [14]:
#Creates first Dataframe from which all proceeding dataframes will be concated to

url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'CompactData/FM/A.GB.'


data = (requests.get(f'{url}{key}{indicator_list.iloc[0, 0]}').json()
        ['CompactData']['DataSet']['Series'])


Great_Britian = pd.DataFrame([
    (pd.to_datetime(i['@TIME_PERIOD']), round(float(i['@OBS_VALUE']), 1))
    for i in data['Obs']
], columns=['Year', 'Cyclically adjusted primary balance (% of potential GDP)'])


Great_Britian['Year'] = Great_Britian['Year'].dt.year

Great_Britian = Great_Britian.set_index('Year')

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [4]:
# Adding all dataframes to existing dataframe for that specific country

count = 0

for item in indicator_list['Indicator'][1:]:    
    
    count += 1
    
    data = (requests.get(f'{url}{key}{item}').json()
        ['CompactData']['DataSet']['Series'])
    
    df = pd.DataFrame([
    (pd.to_datetime(i['@TIME_PERIOD']), round(float(i['@OBS_VALUE']), 1))
    for i in data['Obs']], columns=['Year', indicator_list.iloc[count]['Description']])
    
    df['Year'] = df['Year'].dt.year

    df = df.set_index('Year')
    
    Great_Britian = pd.concat([Great_Britian, df], axis=1)

# Normalizing Defaults Data

In [28]:
defaults = pd.read_excel('GlobalCrisisDataset.xlsx')

In [30]:
defaults = defaults[['Country', 'Year', 'SOVEREIGN EXTERNAL DEBT 2: DEFAULT and RESTRUCTURINGS, 1800-2012--Does not include defaults on WWI debt to United States and United Kingdom but includes post-1975 defaults on Official External Creditors']]


In [32]:
defaults = defaults.set_index('Country')
defaults.columns = ['Year', 'Sovereign Debt Default']

In [40]:
dfs = {}
for group_name, group_data in defaults.groupby(defaults.index):
    dfs[group_name] = group_data.copy()
    
for group_name, group_df in dfs.items():
    print(f"DataFrame for group {group_name}:")
    print(group_df)
    print("-----------------------")    

DataFrame for group Algeria:
           Year  Sovereign Debt Default
Country                                
Algeria  1800.0                     0.0
Algeria  1801.0                     0.0
Algeria  1802.0                     0.0
Algeria  1803.0                     0.0
Algeria  1804.0                     0.0
...         ...                     ...
Algeria  2012.0                     0.0
Algeria  2013.0                     0.0
Algeria  2014.0                     0.0
Algeria  2015.0                     0.0
Algeria  2016.0                     0.0

[217 rows x 2 columns]
-----------------------
DataFrame for group Angola:
           Year  Sovereign Debt Default
Country                                
Angola   1800.0                     0.0
Angola   1801.0                     0.0
Angola   1802.0                     0.0
Angola   1803.0                     0.0
Angola   1804.0                     0.0
...         ...                     ...
Angola   2012.0                     0.0
Angola   2013.0

           Year  Sovereign Debt Default
Country                                
Zambia   1800.0                     0.0
Zambia   1801.0                     0.0
Zambia   1802.0                     0.0
Zambia   1803.0                     0.0
Zambia   1804.0                     0.0
...         ...                     ...
Zambia   2012.0                     0.0
Zambia   2013.0                     0.0
Zambia   2014.0                     0.0
Zambia   2015.0                     0.0
Zambia   2016.0                     0.0

[217 rows x 2 columns]
-----------------------
DataFrame for group Zimbabwe:
            Year  Sovereign Debt Default
Country                                 
Zimbabwe  1800.0                     0.0
Zimbabwe  1801.0                     0.0
Zimbabwe  1802.0                     0.0
Zimbabwe  1803.0                     0.0
Zimbabwe  1804.0                     0.0
...          ...                     ...
Zimbabwe  2012.0                     1.0
Zimbabwe  2013.0                 

In [147]:
crisis_GB = pd.DataFrame(dfs.get('United Kingdom'))

In [149]:
crisis_GB['Year'] = crisis_GB['Year'].astype(int)
crisis_GB['Year'] = pd.to_datetime(crisis_GB['Year'], format='%Y')
crisis_GB['Year'] = crisis_GB['Year'].dt.year
crisis_GB = crisis_GB.set_index('Year')


# Merging crisis & Indicator data

In [150]:
final = pd.concat([Great_Britian, crisis_GB], axis=1)

In [151]:
final.head(30)

,Cyclically adjusted primary balance (% of potential GDP),Cyclically adjusted primary balance (% of potential GDP),Expenditure (% of GDP),Gross debt (% of GDP),Net debt (% of GDP),Net lending/borrowing (also referred as overall balance) (% of GDP),Primary net lending/borrowing (also referred as primary balance) (% of GDP),Revenue (% of GDP),Sovereign Debt Default
Year,,,,,,,,,
1991,-3.8,-2.5,35.7,28.4,24.6,-2.6,-1.4,33.1,0.0
1992,-5.0,-3.7,37.8,33.2,26.1,-5.5,-4.1,32.4,0.0
1993,-5.9,-4.5,37.8,37.9,30.3,-6.7,-5.3,31.1,0.0
1994,-5.4,-3.8,37.5,40.7,35.3,-5.7,-4.1,31.7,0.0
1995,-4.3,-2.4,37.6,43.6,39.2,-5.0,-3.0,32.6,0.0
1996,-2.1,-0.2,35.6,43.7,42.2,-3.5,-1.6,32.0,0.0
1997,-0.9,1.2,34.7,43.2,39.3,-2.0,0.2,32.8,0.0
1998,0.5,2.6,34.3,40.8,37.2,-0.3,1.8,34.0,0.0
1999,1.5,3.2,33.9,39.3,35.9,0.6,2.4,34.5,0.0


In [152]:
final.to_csv('Data/GB.csv')